In [1]:
import open3d as o3d
import os
path = "../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Reading the point cloud using PCD
pcd = o3d.io.read_point_cloud("../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply")
print(pcd)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] Read PLY failed: unable to read file: ../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply
PointCloud with 757691 points.


In [6]:
content = dir(pcd)
# content of the pcd Class
content

['HalfEdgeTriangleMesh',
 'Image',
 'LineSet',
 'PointCloud',
 'RGBDImage',
 'TetraMesh',
 'TriangleMesh',
 'Type',
 'Unspecified',
 'VoxelGrid',
 '__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'cluster_dbscan',
 'colors',
 'compute_convex_hull',
 'compute_mahalanobis_distance',
 'compute_mean_and_covariance',
 'compute_nearest_neighbor_distance',
 'compute_point_cloud_distance',
 'covariances',
 'create_from_depth_image',
 'create_from_rgbd_image',
 'crop',
 'dimension',
 'estimate_covariances',
 'estimate_normals',
 'estimate_point_covariances',
 'get_axis_aligned_bounding_box',
 'get_center',
 'get_geometry_type',
 'get_max_bound'

### Function to downsample input pointcloud into output pointcloud uniformly. The sample is performed in the order of the points with the 0-th point always chosen, not at random.

In [3]:
sampled=pcd.uniform_down_sample(100)
o3d.visualization.draw_geometries([sampled])
sampled

PointCloud with 7577 points.

### Function to downsample input pointcloud into output pointcloud randomly. The sample is generated by randomly sampling the indexes from the point cloud.

In [11]:
sampled=pcd.random_down_sample(0.01)
o3d.visualization.draw_geometries([sampled])
sampled

PointCloud with 7576 points.

### Farthest Point sampling ( FPS )

In [42]:
def fps(points, n_samples):
    """
    points: [N, 3] array containing the whole point cloud
    n_samples: samples you want in the sampled point cloud typically << N 
    """
    # Represent the points by their indices in points
    points_left = np.arange(len(points)) # [P]

    # Initialise an array for the sampled indices
    sample_inds = np.zeros(n_samples, dtype='int') # [S]

    # Initialise distances to inf
    dists = np.ones_like(points_left) * float('inf') # [P]

    # Select a point from points by its index, save it
    selected = 0
    sample_inds[0] = points_left[selected]

    # Delete selected 
    points_left = np.delete(points_left, selected) # [P - 1]

    # Iteratively select points for a maximum of n_samples
    for i in range(1, n_samples):
        # Find the distance to the last added point in selected
        # and all the others
        last_added = sample_inds[i-1]
        
        dist_to_last_added_point = (
            (points[last_added] - points[points_left])**2).sum(-1) # [P - i]

        # If closer, updated distances
        dists[points_left] = np.minimum(dist_to_last_added_point, 
                                        dists[points_left]) # [P - i]

        # We want to pick the one that has the largest nearest neighbour
        # distance to the sampled points
        selected = np.argmax(dists[points_left])
        sample_inds[i] = points_left[selected]

        # Update points_left
        points_left = np.delete(points_left, selected)

    return points[sample_inds]

In [45]:
pcd = o3d.io.read_point_cloud(path)
pcd_np = pcd_xyz = np.asarray(pcd.points)
points = fps(pcd_np,1000)


[Open3D WARNING] Read PLY failed: unable to read file: ../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply


In [46]:
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud("sampled.ply", pcd2)

    # Load saved point cloud and visualize it
pcd_load = o3d.io.read_point_cloud("sampled.ply")
o3d.visualization.draw_geometries([pcd_load])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [50]:
!python "../recolor/pcc_geo_cnn_v2/src/map_color.py" "../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply" "sampled.ply" "sampled_colored.ply"

2022-06-16 15:15:11.687 INFO utils - _init_num_threads: NumExpr defaulting to 4 threads.
C:\Users\Skander\Point clouds\Experimentation\recolor\pcc_geo_cnn_v2\src\map_color.py:33: DeprecationWarning: The n_jobs argument has been renamed "workers". The old name "n_jobs" will stop working in SciPy 1.8.0.
  mapped_indices = kdtree.query(target_pc.points[['x', 'y', 'z']].values, k=2, n_jobs=-1)[1][:, 1]


In [51]:
pcd_load = o3d.io.read_point_cloud("sampled_colored.ply")
o3d.visualization.draw_geometries([pcd_load])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [4]:
!pip install PyntCloud

### Dividing the Point cloud into blocks : 

In [2]:
from pyntcloud import PyntCloud
pc = PyntCloud.from_file(path)
points = pc.points.values
points

array([[292.,  15., 195.,  47.,  34., 104.],
       [290.,  15., 198.,  46.,  34., 107.],
       [290.,  15., 199.,  47.,  35., 108.],
       ...,
       [549., 895., 313.,  82.,  66., 101.],
       [554., 892., 312.,  83.,  67., 102.],
       [554., 893., 312.,  83.,  67., 102.]])

In [25]:
cubes = {}# {block start position, points in block}
point_cloud=points
for _, point in enumerate(point_cloud):
    cube_index = tuple((point//64).astype("int"))
    local_point = point % 64
    if not cube_index in cubes.keys():
        cubes[cube_index] = local_point
    else:
        cubes[cube_index] = np.vstack((cubes[cube_index] ,local_point))
  # filter by minimum number.
k_del = []

In [27]:
for _, k in enumerate(cubes.keys()):
    if cubes[k].shape[0] < 2048:
        k_del.append(k)

In [29]:
for _, k in enumerate(k_del):
    del cubes[k]

In [31]:
cube_positions = np.array(list(cubes.keys()))
set_points = []

In [32]:
step = cube_positions.max() + 1
step

16

In [55]:
cube_positions_n = cube_positions[:,0:1] + cube_positions[:,1:2]*step + cube_positions[:,2:3]*step*step
cube_positions_n = np.sort(cube_positions_n, axis=0)

In [54]:
x = cube_positions_n % step
y = (cube_positions_n // step) % step
z = cube_positions_n // step // step

In [ ]:
cube_positions_orderd = np.concatenate((x,y,z), -1)
for _, k in enumerate(cube_positions_orderd):
    print(tuple(k))
    set_points.append(cubes[tuple(k)].astype("int16"))